# Using EAGO's basic optimizer with user-defined subroutines 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

### Overview  
In this section, we construct an optimizer that uses EAGO's basic nlp solution routine with user-defined lower and upper bounding problems. The **EAGO.Optimizer** structure supplies a number of parameters and stored structures that advanced users may find useful for constructing specialized solution routines. For a full review, of the EAGO.optimizer object the reader is directed to the **EAGO.Optimizer** docstring and documentation provided at [https://psorlab.github.io/EAGO.jl/stable/](https://psorlab.github.io/EAGO.jl/stable/).

In this example, we'll forgo extensive integration into the EAGO.optimizer and simply replace the lower and upper-bounding problems to construct B&B routine that solves the following problem to global optimality using bounds obtained from interval arithmetic:

$
\begin{align}
&\min_{\mathbf x \in X} \;\; \sin(x_1)x_2^2 - \cos(x_3) / x_4 \label{eq1} \\
&X = [-10,10]\times[-1,1]\times[-10,10]\times[2,20].
\end{align}
$

We begin importing EAGO, IntervalArithmetic[1], and JuMP[2].

In [1]:
using EAGO, IntervalArithmetic, JuMP

┌ Info: Recompiling stale cache file C:\Users\wilhe\.julia\compiled\v1.0\EAGO\t0N0c.ji for EAGO [bb8be931-2a91-5aca-9f87-79e1cb69959a]
└ @ Base loading.jl:1190
┌ Warning: Package EAGO does not have CPLEX in its dependencies:
│ - If you have EAGO checked out for development and have
│   added CPLEX as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with EAGO
└ Loading CPLEX into EAGO from project dependency, future warnings for EAGO are suppressed.
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\S

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:2
  likely near C:\Users\wilhe\.julia\dev\

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:16
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe

  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_operations.jl:41
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_utils.jl:2
  likely near C:\Users\wilhe\.julia\dev\EAGO\src\Script\execute_utils.jl:3
  likely near C:\Users\wilhe\.julia\dev\

### Defining a custom lower bounding problem
A valid lower bound is obtained from the lower bound of the natural interval extension using the **ValidatedNumerics.jl** package. The LowerProblem used accepts the **EAGO.Optimizer** structure and a **EAGO.NodeBB** structure, computes the bound by method overloading interval arithmetic, and stores the results to the appropriate field of the **EAGO.Optimizer's** which is a mutable structure of type **EAGO.LowerInfo**. Note that the problem is unconstrained on the domain so we can assume it is always feasible.

In [2]:
function LowerProblem!(opt::EAGO.Optimizer, n::EAGO.NodeBB)
    x = Interval.(n.lower_variable_bounds,n.upper_variable_bounds)
    FInterval = sin(x[1])x[2]^2-cos(x[3])/x[4]
    opt.current_lower_info.value = FInterval.lo
    opt.current_lower_info.solution = mid.(x)
    opt.current_lower_info.feasibility = true
end

LowerProblem! (generic function with 1 method)

### Defining a custom upper bounding problem
Since the problem is unconstained, any feasible point represents a valid upper bound. If we arbitrarily take evaluate the function at the midpoint we obtain an upper bound. If an interval extension of the midpoint is used this upper bound is guaranteed to be valid inspite of any rounding error that may occur. The below function constructs an upper bound in this manner then stores the results to the appropriate field of the **EAGO.Optimizer's** which is a mutable structure of type **EAGO.UpperInfo**.

In [3]:
function UpperProblem!(opt::EAGO.Optimizer, n::EAGO.NodeBB)
    x_value = (n.upper_variable_bounds-n.lower_variable_bounds)/2.0
    x_interval = Interval.(x_value)
    FInterval = sin(x_interval[1])x_interval[2]^2-cos(x_interval[3])/x_interval[4]
    opt.current_upper_info.value = FInterval.hi
    opt.current_upper_info.solution = x_value
    opt.current_upper_info.feasibility = true
end

UpperProblem! (generic function with 1 method)

### Build the JuMP Model and optimize
We now add our optimizer to a JuMP model, provide variable bounds, user-defined functions, and optimize. Note that options can be provided to the EAGO optimizer using a series of keywords of a Dict{Symbol,Any} object. Both manners of providing options to EAGO are illustrated below.

In [6]:
# Creates a JuMP model with the the lower_problem, upper_problem, and absolute tolerance set by keyword arguments
m = JuMP.Model(with_optimizer(EAGO.Optimizer, lower_problem! = LowerProblem!, upper_problem! = UpperProblem!, 
                                               absolute_tolerance = 0.001))

# Create the same model m using an options dictionary
opt_dict = Dict{Symbol, Any}()
opt_dict[:lower_problem!] = LowerProblem!
opt_dict[:upper_problem!] = UpperProblem!
opt_dict[:absolute_tolerance] = 0.001

m = JuMP.Model(with_optimizer(EAGO.Optimizer; opt_dict...))

# Adds variables and bounds
x_L = [-10, -1, -10, 2]
x_U = [10, 1, 10, 20]
@variable(m, x_L[i] <= x[i=1:4] <= x_U[i])

# Solves the problem
JuMP.optimize!(m)

start set to default
end set to default
L: -Inf, U: Inf
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 1, Lower Bound: -Inf, Lower Variable Bounds:
             [-10.0, -1.0, -10.0, 2.0], Upper Variable Bounds: [10.0, 1.0, 10.0, 20.0]
started initial preprocessing
finished initial preprocessing
FInterval lower: [-1.5, 1.5]finished initial lower problem
FInterval upper: [-0.450791, -0.45079]finished initial upper problem
finished initial postprocessing
FInterval lower: [-1.5, 1.5]Lower Bound: -1.5, Solution: [0.0, 0.0, 0.0, 11.0], Feasibility: true
FInterval upper: [-0.450791, -0.45079]Upper Bound: -0.45079094099198613, Solution: [10.0, 1.0, 10.0, 9.0], Feasibility: true
Iteration   NodeID    Current_LBD     Global_LBD     Global_UBD      NodesLeft     Absolute_Gap    Absolute_Ratio     LBD_Feas     UBD_Feas
     1,        2,      -1.5000000,     -Inf,     -0.4507909,           2,        Inf,       Inf,        true,       tru

FInterval lower: [-1.5, 1.5]Lower Bound: -1.5, Solution: [2.42172, 0.496094, 4.96094, 6.46484], Feasibility: true
FInterval upper: [0.0801246, 0.0801247]Upper Bound: 0.08012466052082606, Solution: [2.49985, 0.503906, 5.03906, 4.46484], Feasibility: true
    16,       17,      -1.5000000,     -1.5000000,     -0.8760403,          17,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: 0.08012466052082606
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 33, Lower Bound: -1.5, Lower Variable Bounds:
             [-0.078125, 0.492157, -0.078125, 2.0], Upper Variable Bounds: [4.92157, 1.0, 10.0, 10.9297]
FInterval lower: [-1.5, 1.5]Lower Bound: -1.5, Solution: [2.42172, 0.746078, 4.96094, 6.46484], Feasibility: true
FInterval upper: [-0.0332763, -0.0332762]Upper Bound: -0.033276277337097794, Solution: [2.49985, 0.253922, 5.03906, 4.46484], Feasibility: true
    17,       18,      -1.5000000,     -1.5000000,     -0

    30,       29,      -1.5000000,     -1.5000000,     -0.8760403,          29,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: -0.15288845863098305
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 60, Lower Bound: -1.5, Lower Variable Bounds:
             [-10.0, -0.0078125, -10.0, 2.0], Upper Variable Bounds: [-0.078125, 1.0, -0.078125, 20.0]
FInterval lower: [-1.5, 1.5]Lower Bound: -1.5, Solution: [-5.03906, 0.496094, -5.03906, 11.0], Feasibility: true
FInterval upper: [-0.273452, -0.273451]Upper Bound: -0.2734516665015934, Solution: [4.96094, 0.503906, 4.96094, 9.0], Feasibility: true
    31,       30,      -1.5000000,     -1.5000000,     -0.8760403,          30,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: -0.2734516665015934
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 62, Lower Bound: -1.5, Lower Variable Bou

FInterval lower: [-1.5, 1.5]Lower Bound: -1.5, Solution: [2.42172, 0.746078, -5.03906, 6.46484], Feasibility: true
FInterval upper: [-0.0165015, -0.0165014]Upper Bound: -0.016501466729676242, Solution: [2.49985, 0.253922, 4.96094, 4.46484], Feasibility: true
    53,       48,      -1.5000000,     -1.5000000,     -0.8760403,          48,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: -0.016501466729676242
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 106, Lower Bound: -1.5, Lower Variable Bounds:
             [-0.078125, 0.492157, -10.0, 2.0], Upper Variable Bounds: [4.92157, 1.0, -5.07782, 10.9297]
FInterval lower: [-1.5, 1.5]Lower Bound: -1.5, Solution: [2.42172, 0.746078, -7.53891, 6.46484], Feasibility: true
FInterval upper: [0.212678, 0.212679]Upper Bound: 0.21267877110526875, Solution: [2.49985, 0.253922, 2.46109, 4.46484], Feasibility: true
    54,       49,      -1.5000000,     -1.5000000,    

    67,       62,      -1.5000000,     -1.5000000,     -0.8760403,          62,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: -0.13118833534088936
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 134, Lower Bound: -1.5, Lower Variable Bounds:
             [-10.0, 0.744095, -7.55814, 2.0], Upper Variable Bounds: [-7.55814, 1.0, -5.07782, 4.19768]
FInterval lower: [-1.5, 0.474567]Lower Bound: -1.5, Solution: [-8.77907, 0.872047, -6.31798, 3.09884], Feasibility: true
FInterval upper: [-0.280065, -0.280064]Upper Bound: -0.2800648868730759, Solution: [1.22093, 0.127953, 1.24016, 1.09884], Feasibility: true
    68,       63,      -1.5000000,     -1.5000000,     -0.8760403,          63,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: -0.2800648868730759
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 136, Lower Bound: -1.5, Lo

    81,       76,      -1.5000000,     -1.5000000,     -0.8760403,          76,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: 0.37145209067599416
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 71, Lower Bound: -1.5, Lower Variable Bounds:
             [-10.0, 0.492157, -5.07782, 2.0], Upper Variable Bounds: [-5.07782, 1.0, -0.078125, 10.9297]
FInterval lower: [-1.49848, 1.43397]Lower Bound: -1.4984748970380144, Solution: [-7.53891, 0.746078, -2.57797, 6.46484], Feasibility: true
FInterval upper: [0.21998, 0.219981]Upper Bound: 0.21998061890355394, Solution: [2.46109, 0.253922, 2.49985, 4.46484], Feasibility: true
    82,       77,      -1.4984749,     -1.5000000,     -0.8760403,          77,        0.6239597,       0.7122499,        true,       true
L: -1.4984748970380144, U: 0.21998061890355394
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node I

FInterval upper: [0.225118, 0.225119]Upper Bound: 0.22511820568776592, Solution: [2.46109, 0.253922, 2.53922, 4.46484], Feasibility: true
    96,       87,      -1.5000000,     -1.5000000,     -0.8760403,          87,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: 0.22511820568776592
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 193, Lower Bound: -1.5, Lower Variable Bounds:
             [-10.0, 0.492157, 4.92157, 6.42996], Upper Variable Bounds: [-5.07782, 1.0, 10.0, 10.9297]
FInterval lower: [-1.15553, 1.0895]Lower Bound: -1.1555219106109722, Solution: [-7.53891, 0.746078, 7.46078, 8.67982], Feasibility: true
FInterval upper: [0.406807, 0.406808]Upper Bound: 0.4068078241793568, Solution: [2.46109, 0.253922, 2.53922, 2.24986], Feasibility: true
    97,       88,      -1.1555219,     -1.5000000,     -0.8760403,          88,        0.6239597,       0.7122499,        true,       true
L: -1.155521910610

L: -1.5, U: -0.2687484579452058
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 220, Lower Bound: -1.5, Lower Variable Bounds:
             [-0.078125, -1.0, -0.078125, 2.0], Upper Variable Bounds: [10.0, -0.0078125, 10.0, 10.9297]
FInterval lower: [-1.5, 1.5]Lower Bound: -1.5, Solution: [4.96094, -0.503906, 4.96094, 6.46484], Feasibility: true
FInterval upper: [-0.304965, -0.304964]Upper Bound: -0.30496486936896405, Solution: [5.03906, 0.496094, 5.03906, 4.46484], Feasibility: true
   111,      102,      -1.5000000,     -1.5000000,     -0.8760403,         102,        0.6239597,       0.7122499,        true,       true
L: -1.5, U: -0.30496486936896405
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 223, Lower Bound: -1.5, Lower Variable Bounds:
             [4.92157, -1.0, -0.078125, 2.0], Upper Variable Bounds: [10.0, -0.0078125, 10.0, 10.9297]
FInterval lower: [-1

             [-10.0, 0.744095, 2.40219, 2.0], Upper Variable Bounds: [-5.07782, 1.0, 4.92157, 6.42996]
FInterval lower: [-1.10383, 1.43397]Lower Bound: -1.1038293349706882, Solution: [-7.53891, 0.872047, 3.66188, 4.21498], Feasibility: true
FInterval upper: [-0.127901, -0.1279]Upper Bound: -0.12790037308257785, Solution: [2.46109, 0.127953, 1.25969, 2.21498], Feasibility: true
   125,      116,      -1.1038293,     -1.5000000,     -0.8760403,         116,        0.6239597,       0.7122499,        true,       true
L: -1.1038293349706882, U: -0.12790037308257785
relative_tolerance 0.0001
absolute_tolerance 0.001
t1: true, t2: true, t3: true, t4: true, t5: true
Node ID: 248, Lower Bound: -1.5, Lower Variable Bounds:
             [-10.0, 0.744095, -0.078125, 2.0], Upper Variable Bounds: [-5.07782, 1.0, 2.40219, 6.42996]
FInterval lower: [-1.5, 1.30341]Lower Bound: -1.5, Solution: [-7.53891, 0.872047, 1.16203, 4.21498], Feasibility: true
FInterval upper: [-0.136268, -0.136267]Upper Bound: -

### Get information from the JuMP Model object
The objective value, solution, termination status,  and primal status can then be accessed via the standard JuMP interface.

In [7]:
fval = JuMP.objective_value(m)
xsol = JuMP.value.(x)
status_term = JuMP.termination_status(m)
status_prim = JuMP.primal_status(m)

println("EAGO terminated with a status of $status_term and a result code of $status_prim")
println("The optimal value is: $fval, the solution found is $xsol.")

EAGO terminated with a status of OPTIMAL and a result code of FEASIBLE_POINT
The optimal value is: -1.4749615097161377, the solution found is [0.624924, 0.0605696, 0.634765, 0.545127].


### Advice for more advanced constructions
The *default_lower_problem* and *default_upper_problem* should be used templates for error handling and retreiving information from MOI models. Similarly, the other default routine are good starting points for building custom modifications.

Essentially all of EAGO's subroutines are stored to a field in the **EAGO.Optimizer** structure can be reset as user-defined functions.

### References
1. IntervalArithmetic.jl [Computer software] (2019). Retrieved from https://github.com/JuliaIntervals/IntervalArithmetic.jl
2. Iain Dunning and Joey Huchette and Miles Lubin. JuMP: A Modeling Language for Mathematical Optimization, *SIAM Review*, **SIAM** 59 (2017), pp. 295-320.